# Задачка с собеседований

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('finance.db')

In [3]:
pd.read_sql("""
    select date, account, value from debit
    """, con)

,date,account,value
0,2021-01-01,111,100
1,2021-01-01,222,200
2,2021-01-01,111,110
3,2021-01-02,111,90
4,2021-01-03,111,90


In [4]:
pd.read_sql("""
    select date, account, value from credit
    """, con)

,date,account,value
0,2021-01-01,111,50
1,2021-01-01,222,100
2,2021-01-01,111,55
3,2021-01-02,111,45


### Ошибочное объединение

In [5]:
pd.read_sql("""
    select d.date, d.account, d.value, c.value from debit d
    left outer join credit c
    on d.date=c.date and d.account = c.account
    """, con)

,date,account,value,value
0,2021-01-01,111,100,50.0
1,2021-01-01,111,100,55.0
2,2021-01-01,222,200,100.0
3,2021-01-01,111,110,50.0
4,2021-01-01,111,110,55.0
5,2021-01-02,111,90,45.0
6,2021-01-03,111,90,NaN


Добавляем группировку

In [7]:
pd.read_sql("""
    select d.date, d.account, d.value from (
        select date, account, sum(value) as value
        from debit
        group by date, account
    ) d
    """, con)

,date,account,value
0,2021-01-01,111,210
1,2021-01-01,222,200
2,2021-01-02,111,90
3,2021-01-03,111,90


In [8]:
pd.read_sql("""
    select c.date, c.account, c.value from (
        select date, account, sum(value) as value
        from credit
        group by date, account
    ) c
    """, con)

,date,account,value
0,2021-01-01,111,105
1,2021-01-01,222,100
2,2021-01-02,111,45


И только после группировки объединяем

In [9]:
pd.read_sql("""
    select d.date, d.account, d.value from (
        select date, account, sum(value) as value
        from debit
        group by date, account
    ) d
    
    left outer join
    
    (select c.date, c.account, c.value from (
        select date, account, sum(value) as value
        from credit
        group by date, account
    ) c) c_2
    
    on d.date=c_2.date and d.account=c_2.account
    """, con)

,date,account,value
0,2021-01-01,111,210
1,2021-01-01,222,200
2,2021-01-02,111,90
3,2021-01-03,111,90


### Вариант получше через UNION

In [10]:
pd.read_sql("""
    select date, account, sum(value) 
    from debit
    group by date, account
    
    union all
    
    select date, account, -sum(value) 
    from credit
    group by date, account
    """, con)

,date,account,sum(value)
0,2021-01-01,111,210
1,2021-01-01,222,200
2,2021-01-02,111,90
3,2021-01-03,111,90
4,2021-01-01,111,-105
5,2021-01-01,222,-100
6,2021-01-02,111,-45


In [11]:
pd.read_sql(""" 
    select date, account, sum(value) from (
        select date, account, sum(value) as value
        from debit
        group by date, account

        union all

        select date, account, -sum(value) as value
        from credit
        group by date, account
    )
    group by date, account
    """, con)

,date,account,sum(value)
0,2021-01-01,111,105
1,2021-01-01,222,100
2,2021-01-02,111,45
3,2021-01-03,111,90


In [12]:
con.close()

# Простая база данных

Запись датафрейма в базу данных

In [13]:
import pandas as pd
import sqlite3

In [14]:
df = pd.read_csv('keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [15]:
con = sqlite3.connect('keywords.db')

In [16]:
df.to_sql('keywords', con, if_exists='replace')

In [17]:
con.close()

Чтение из базы

In [18]:
con = sqlite3.connect('keywords.db')

In [19]:
df_sql = pd.read_sql('select * from keywords where keyword like "%вконтакте%";', con)
df_sql

,index,keyword,shows
0,4,вконтакте,21014195
1,14,вконтакте моя страница,5971451
2,317,вконтакте вход на страницу,488442
3,530,моя страница вконтакте,5971451
4,896,вконтакте социальная сеть,202480
...,...,...,...
84,96819,оренбург онлайн вконтакте,3894
85,97029,раскрутка групп в вконтакте,3782
86,98809,весь арзамас вконтакте,3623
87,99696,моя страница вконтакте одноклассники,16548


Построчная обработка

In [20]:
cur = con.cursor()

In [21]:
cur.execute('select * from keywords where keyword like "%вконтакте%";')

In [22]:
line = cur.fetchone()
line

(4, 'вконтакте', 21014195)

In [23]:
another_line = cur.fetchone()
another_line

(14, 'вконтакте моя страница', 5971451)

In [24]:
for i, line in enumerate(cur.execute('select * from keywords where keyword like "%вконтакте%";')):
    print(line)
    
    if i > 5:
        break

(4, 'вконтакте', 21014195)
(14, 'вконтакте моя страница', 5971451)
(317, 'вконтакте вход на страницу', 488442)
(530, 'моя страница вконтакте', 5971451)
(896, 'вконтакте социальная сеть', 202480)
(1003, 'вконтакте вход', 190587)
(1093, 'вконтакте моя', 173001)
